In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime

# Define your API key
api_key = '17b993e3-1e66-445c-989b-7b6f6f12c046'

def get_current_btc_price(api_key):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
    parameters = {
        'symbol': 'BTC',
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    response = requests.get(url, headers=headers, params=parameters)
    data = response.json()
    btc_price = data['data']['BTC']['quote']['USD']['price']
    return btc_price

def compute_rsi(data, window=14):
    delta = data.diff(1).dropna()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Load the new historical data
file_path = 'historical_data.csv'
historical_data = pd.read_csv(file_path)

# Convert 'Date' column to datetime and set as index
historical_data['Date'] = pd.to_datetime(historical_data['Date'], format='%m/%d/%Y')
historical_data.set_index('Date', inplace=True)

# Convert price-related columns to numeric and 'Vol.' to float
historical_data['Price'] = historical_data['Price'].str.replace(',', '').astype(float)
historical_data['Open'] = historical_data['Open'].str.replace(',', '').astype(float)
historical_data['High'] = historical_data['High'].str.replace(',', '').astype(float)
historical_data['Low'] = historical_data['Low'].str.replace(',', '').astype(float)

# Convert 'Vol.' to numeric (assuming 'K' stands for thousand)
historical_data['Vol.'] = historical_data['Vol.'].str.replace('K', '').astype(float) * 1000

# Calculate RSI and Moving Average
historical_data['RSI'] = compute_rsi(historical_data['Price'])
historical_data['Moving_Average'] = historical_data['Price'].rolling(window=50).mean()

# Verbose output to check data after conversion
print(historical_data.head())

# Drop rows with NaN values created by rolling function
historical_data.dropna(inplace=True)

# Verbose output to check data after dropping NaN values
print(historical_data.head())

# If historical_data is empty, raise an informative error
if historical_data.empty:
    raise ValueError('No data available after processing. Check data conversions and dropna operations.')

# Define a target variable for machine learning
historical_data['Future_Price'] = historical_data['Price'].shift(-1)
historical_data.dropna(inplace=True)

# Verbose output to check data before splitting
print(historical_data.head())

# Split data into features and target for machine learning
X = historical_data[['Price', 'Vol.', 'Moving_Average', 'RSI']]
y = historical_data['Future_Price']

# Check if X or y are empty
if X.empty or y.empty:
    raise ValueError('Features (X) or target (y) are empty after processing. Check data integrity and target variable creation.')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Model training
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Model prediction
predictions = model.predict(X_test)

# Evaluate model (implement an evaluation metric)
print("Model Accuracy: ", model.score(X_test, y_test))

# Strategy implementation (simplified example)
def should_sell(current_price, ath_price, threshold=0.05):
    return current_price < ath_price * (1 - threshold)

# Example usage
current_price = get_current_btc_price(api_key)
ath_price = historical_data['Price'].max()

if should_sell(current_price, ath_price):
    print("It's time to sell")
else:
    print("Hold")

# Plotting
plt.style.use('dark_background')

# Plotting Price and Moving Average
plt.figure(figsize=(12, 6))
plt.plot(historical_data.index, historical_data['Price'], label='BTC Close Price', alpha=0.5)
plt.plot(historical_data.index, historical_data['Moving_Average'], label='50-Day Moving Average', alpha=0.7)
plt.title('BTC Historical Prices with Moving Average')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-rotate dates
plt.show()

# Plotting RSI
plt.figure(figsize=(12, 4))
plt.plot(historical_data.index, historical_data['RSI'], label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--', alpha=0.5)  # Overbought line
plt.axhline(30, color='green', linestyle='--', alpha=0.5)  # Oversold line
plt.title('Relative Strength Index (RSI)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-rotate dates
plt.show()

# Plotting Model Predictions vs Actual Prices
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual Future Price', alpha=0.7)
plt.plot(y_test.index, predictions, label='Predicted Future Price', alpha=0.7, linestyle='--')
plt.title('Model Predictions vs Actual Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-rotate dates
plt.show()


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.